In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

print(np.__version__)
print(pd.__version__)
import sys
print(sys.version)
print(sns.__version__)

In [ ]:
df = pd.read_csv("D:\\repos\\APDSProject\\MLTuning\\datacleanup\\cvss_final_dataset.csv")
df.head()
df.shape
#df.info()
#df.columns



# Remove Unwanted Columns

In [ ]:

remove_list = ['timestamp','data_type','data_format', 'data_version','data_meta_ASSIGNER','cvssV3_version','cvssV3_vectorString','cvssV2_version','cvssV2_vectorString']

df.drop(remove_list,axis = 1, inplace = True)
df.info
df.shape



# Prepare CVSS3 Data Frame




In [ ]:
df.columns
cvss3_features = ['cvssV3_attackVector', 'cvssV3_attackComplexity', 'cvssV3_privilegesRequired', 'cvssV3_userInteraction', 'cvssV3_scope','cvssV3_confidentialityImpact', 'cvssV3_integrityImpact','cvssV3_availabilityImpact', 'cvssV3_baseScore', 'cvssV3_baseSeverity',      'baseMetricV3_exploitabilityScore', 'baseMetricV3_impactScore', ]

cvss3_df = df[cvss3_features]

cvss3_df.columns
cvss3_df.info
cvss3_df.shape


# Extract the numerical and categorical columns from CVSS Version 3 Featue Set

In [ ]:
cvssv3_numerical_col = list(cvss3_df.describe().columns)
cvssv3_categorical_col = list(set(cvss3_df.columns).difference(cvssv3_numerical_col))

cvssv3_numerical_col


In [ ]:
cvss3_df[cvssv3_numerical_col].corr()
plt.figure(figsize=(24,8))
sns.heatmap(cvss3_df[cvssv3_numerical_col].corr(), annot=True, fmt=".2f");

In [ ]:
cvssv3_categorical_col

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
cvss3_df.describe()
cvss3_df.info()

# Prepare the features for modelling

In [ ]:

cvssv3_model_df= cvss3_df[cvssv3_categorical_col]
cvssv3_model_df.columns

cvssv3_model_df.info

In [ ]:
cvssv3_model_df.isnull().any()

# Perform One Hot Encoding
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

In [ ]:
df_v3_model = pd.get_dummies(cvssv3_model_df[cvssv3_categorical_col])


In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df_v3_model.describe()
df_v3_model.info

In [ ]:
plt.figure(figsize=(24,8))
sns.heatmap(df_v3_model.corr(), annot=True, fmt=".2f");

In [ ]:
import numpy as np

# Create correlation matrix
corr_matrix = df_v3_model.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]


# Drop features 
df_v3_model.drop(to_drop, axis=1, inplace=True)
to_drop

In [ ]:
#X = pd.concat([df['cvssV3_baseScore'], df_v3_model], axis=1)
X = df_v3_model
X.columns

In [ ]:
plt.figure(figsize=(24,8))

# Mask for the upper triangle
mask = np.zeros_like(X.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(10, 220, as_cmap=True)
cmap = sns.light_palette((210, 90, 60), input="husl")

# Heatmap with mask
sns.heatmap(X.corr(), mask=mask, cmap=cmap, annot=True, fmt=".2f");

In [ ]:
correlatedColumns = []
corr = X.corr()
indices = corr.index
columns = corr.columns
posthreshold = 0.7
negthreshold = -0.7

for c in columns:
    for r in indices:
        if c != r and (corr[c][r] > posthreshold or corr[c][r] < negthreshold):
            print("column "  + c , "  row " + r + "  val " + str(corr[c][r]) )
            correlatedColumns.append({"column" : c , "row" : r , "val" :corr[c][r] })
            

print(correlatedColumns)

In [ ]:
def calculateSeverityScore(baseScore):
    #baseScore = score
    if baseScore < 10.0 and baseScore > 8.0:
        return 'CRITICAL'
    elif baseScore < 8.0 and baseScore > 6.0:
        return 'HIGH'
    elif baseScore < 6.0 and baseScore > 4.0:
        return 'MEDIUM'
    else:
        return 'LOW'

#severity = calculateSeverityScore(cvss3_df.iloc[0]['cvssV3_baseScore'])
#severity

cvss3_df['Severity_Score'] = cvss3_df['cvssV3_baseScore'].apply(calculateSeverityScore)


In [ ]:
cvss3_df['Severity_Score'].unique()

In [ ]:
severity_to_num = {'CRITICAL': 1,
                    'HIGH': 2,
                    'MEDIUM': 3,
                    'LOW': 4}
cvss3_df['Severity_Score_Num'] = cvss3_df['Severity_Score'].map(severity_to_num)

In [ ]:
cvss3_df['Severity_Score_Num'].unique()


In [ ]:
cvss3_df.rename(columns = {'Severity_Score':'Severity_Score_Text'}, inplace = True)


In [ ]:
cvss3_df.columns

In [ ]:
cvss3_df.rename(columns = {'Severity_Score_Num':'Severity_Score'}, inplace = True)
Y = cvss3_df['Severity_Score']
Y.head


In [ ]:
X.to_csv("features.csv", index=False, encoding='utf8')


In [ ]:
Y.to_csv("classes.csv",index=False,encoding='utf8')

In [ ]:
X.head

In [ ]:
X['CVSS3_Base_Score']=cvss3_df['cvssV3_baseScore']

In [ ]:
Y.head

In [ ]:
X.to_csv("features_vineet.csv", index=False, encoding='utf8')

# Split the data set into 70% training and 30% test 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1) # 70% training and 30% test

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
clf.fit(X_train, y_train)



In [ ]:
from sklearn.tree.export import export_text
tree_rules = export_text(clf, feature_names = list(X.columns))
tree_rules

In [ ]:
def get_code(tree, feature_names):
        left      = tree.tree_.children_left
        right     = tree.tree_.children_right
        threshold = tree.tree_.threshold
        features  = [feature_names[i] for i in tree.tree_.feature]
        value = tree.tree_.value

        def recurse(left, right, threshold, features, node):
                if (threshold[node] != -2):
                        print ("if ( " + features[node] + " <= " + str(threshold[node]) + " ) {")
                        if left[node] != -1:
                                recurse (left, right, threshold, features,left[node])
                        print ("} else {")
                        if right[node] != -1:
                                recurse (left, right, threshold, features,right[node])
                        print ("}")
                else:
                        print ("return " + str(value[node]))

        recurse(left, right, threshold, features, 0)



In [ ]:
    get_code(clf,list(X.columns)) | samp.txt

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy_score(y_train, clf.predict(X_train))

In [ ]:
print(classification_report(y_train, clf.predict(X_train)))

In [ ]:
confusion_matrix(y_train, clf.predict(X_train))

In [ ]:
accuracy_score(y_test, clf.predict(X_test))

In [ ]:
print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
confusion_matrix(y_test, clf.predict(X_test))

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        res = clf.predict(X_train)
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, 
                                                                res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, 
                                                                            res)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, 
                                                                  res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                      lb.transform(res))))

        #res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, 
                                                                res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, 
                                                                            res_test)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, 
                                                                  res_test)))   
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), 
                                                      lb.transform(res_test))))

In [ ]:
print_score(clf, X_train, X_test, y_train, y_test, train=True)
print_score(clf, X_train, X_test, y_train, y_test, train=False)

In [ ]:
from sklearn import tree

text_representation = tree.export_text(clf)
print(text_representation)

In [218]:
import requests
import pandas as pd

url = 'http://www.cvedetails.com/vulnerability-list/vendor_id-1/product_id-4'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1]
#print(df)
new_df = df.iloc[::2]
new_df.to_csv('my data.csv')